In [1]:
import numpy as np
from numpy.random import random
from copy import  deepcopy

# Load input


In [2]:
from envyron.io.input import Input

In [3]:
natoms = 3
my_input = Input(natoms=natoms, filename='input.yml')
my_input.ions

IonsModel(atomicspread=[0.5, 0.5, 0.5], corespread=[0.5, 0.5, 0.5], solvationrad=[0.0, 0.0, 0.0])

# Initialize a cell


In [4]:
from envyron.domains.cell import EnvironGrid

In [5]:
L = 20.

at = np.array([
    [L, 0., 0.],
    [0., L, 0.],
    [0., 0., L],
])

nr = np.array([50, 50, 50])

grid = EnvironGrid(at, nr, label='system')
print(grid.origin)
print(grid.lattice)
print(grid.volume)

[0. 0. 0.]
[[20.  0.  0.]
 [ 0. 20.  0.]
 [ 0.  0. 20.]]
8000.0


# Compute minimum image distances


In [6]:
r, r2 = grid.get_min_distance(grid.origin)

# Initialize a density


In [7]:
from envyron.representations import EnvironDensity

In [10]:
density = EnvironDensity(grid, data=random(nr), label='electrons')
print(density.label)
# density.standard_view()

electrons


# Compute multipoles


In [11]:
origin = np.ones(3) * 0.5
density.compute_multipoles(origin)
print(f"charge = {density.charge}")
print(f"dipole = {density.dipole}")
print(f"quadrupole = {density.quadrupole}")

charge = 3996.5189848201458
dipole = [418.15705082 368.27232821 399.67207504]
quadrupole = [133054.27046891 133124.82012506 133650.23863224]


# Compute euclidean norm


In [ ]:
en = density.euclidean_norm()
# en

# Compute quadratic mean


In [ ]:
qm = density.quadratic_mean()
# qm

# Compute scalar product


In [ ]:
other_density = EnvironDensity(grid, data=random(nr))
sp = density.scalar_product(other_density)
# sp

# Initialize a gradient


In [ ]:
from envyron.representations import EnvironGradient

In [ ]:
gradient = EnvironGradient(grid, data=random((3, *nr)), label='electrons')
print(gradient.label)
# gradient.standard_view()

# Compute modulus


In [ ]:
mod = gradient.modulus
print(mod.label)
# mod.standard_view()

# Compute scalar products


In [ ]:
other_gradient = EnvironGradient(grid, data=random((3, *nr)))
sgp = gradient.scalar_gradient_product(other_gradient)
# sgp.standard_view()

In [ ]:
sdp = gradient.scalar_density_product(density)
# sdp

# Initialize a hessian


In [ ]:
from envyron.representations import EnvironHessian

In [ ]:
hessian = EnvironHessian(grid, data=random((9, *nr)), label='boundary')
print(hessian.label)
# hessian.standard_view()

# Compute laplacian


In [ ]:
lapl = hessian.trace
print(lapl.label)
# hessian.laplacian.standard_view()

# Compute scalar product


In [ ]:
sgp = hessian.scalar_gradient_product(gradient)
# sgp.standard_view()

# Initialize a gaussian function


In [ ]:
from envyron.representations.functions import EnvironGaussian

In [ ]:
gaussian = EnvironGaussian(grid, 1, 0, 0, 0.5, 0.5, 0.5, np.array([5, 5, 5]), 'ions')
gaussian.kind

# Compute density of function


In [ ]:
gd = gaussian.density
print(gd.label)
print(gd.shape)
print(gd.size)
print(gd[gd != 0].size)

# Compute gradient of function


In [ ]:
gg = gaussian.gradient
print(gg.label)
print(gg.shape)
print(gg.size)
print(gg[gg != 0].size)

# Initialize an erfc function


In [ ]:
from envyron.representations.functions import EnvironERFC

In [ ]:
erfc = EnvironERFC(grid, 2, 0, 0, 0.5, 0.5, 0.5, np.array([5, 5, 5]), 'soft_spheres')
erfc.kind

# Compute density of function


In [ ]:
ed = erfc.density
print(ed.label)
print(ed.shape)
print(ed.size)
print(ed[ed != 0].size)

# Compute gradient of function


In [ ]:
eg = erfc.gradient
print(eg.label)
print(eg.shape)
print(eg.size)
print(eg[eg != 0].size)

# Compute laplacian of function


In [ ]:
el = erfc.laplacian
print(el.label)
print(el.shape)
print(el.size)
print(el[el != 0].size)

# Compute hessian of function


In [ ]:
eh = erfc.hessian
print(eh.label)
print(eh.shape)
print(eh.size)
print(eh[eh != 0].size)

# Compute derivative of function


In [ ]:
edv = erfc.derivative
print(edv.label)
print(edv.shape)
print(edv.size)
print(edv[edv != 0].size)

# Initialize ions

In [ ]:
from envyron.physical import EnvironIons

In [ ]:
ions = EnvironIons(
    nions=natoms,
    ntypes=2,
    itypes=[0, 1, 1],
    ion_ids=['O', 'H'],
    zv=[6.0, 1.0],
    atomicspread=my_input.ions.atomicspread,
    corespread=my_input.ions.corespread,
    solvationrad=my_input.ions.solvationrad,
    radius_mode=my_input.solvent.radius_mode,
    is_soft_cavity=False,
    smear=True,
    fill_cores=True,
    grid=grid,
)

In [ ]:
coords = np.array([
    [11.79, 12.05, 11.50],
    [13.45, 11.22, 11.50],
    [10.56, 10.66, 11.50],
])

In [ ]:
for ion in ions.smeared_ions:
    print(
        ion.label,
        ion.kind,
        ion.dim,
        ion.axis,
        ion.width,
        ion.spread,
        ion.volume,
        ion.pos,
    )

In [ ]:
ions.iontypes[1].solvationrad

In [ ]:
ions.update(coords)

In [ ]:
for ion in ions.smeared_ions:
    print(ion.pos)

In [ ]:
print(f"charge     = {ions.charge}")
print(f"center     = {ions.com}")
print(f"quad corr  = {ions.quadrupole_correction}")
print(f"quad pc    = {ions.quadrupole_pc}")
print(f"quad gauss = {ions.quadrupole_gauss}")
print(f"pot shift  = {ions.potential_shift}")

In [ ]:
d = ions.density

In [ ]:
sd = ions.smeared_ions.density()

In [ ]:
sd = ions.smeared_ions[[0, 2]].density()

In [ ]:
sd = ions.smeared_ions[0].density

# Initialize system

In [ ]:
from envyron.physical import EnvironSystem

In [ ]:
system = EnvironSystem(
    ntypes=1,
    dim=0,
    axis=2,
    ions=ions,
)

In [ ]:
system.ntypes

In [ ]:
system.update()

In [ ]:
system.com

# Initialize electrons

In [ ]:
from envyron.physical import EnvironElectrons

In [ ]:
electrons = EnvironElectrons(grid)

In [ ]:
rho = random(nr)

In [ ]:
electrons.update(rho / 1000)
print(f"charge = {electrons.charge}")
print(f"count  = {electrons.count}")

# Initialize a core container

In [ ]:
from envyron.cores import CoreContainer

In [ ]:
cores = CoreContainer(label='outer', has_internal_correction=False)

# Initialize an electronic boundary

In [ ]:
from envyron.boundaries import ElectronicBoundary

In [ ]:
eboundary = ElectronicBoundary(
    rhomin=my_input.solvent.rhomin,
    rhomax=my_input.solvent.rhomax,
    electrons=electrons,
    mode='electronic',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='chain',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [ ]:
# electrons.updating = True
eboundary.update()
# electrons.updating = False

In [ ]:
print(f"volume  = {eboundary.volume}")
print(f"surface = {eboundary.surface}")

# Initialize an ionic boundary

In [ ]:
from envyron.boundaries import IonicBoundary

In [ ]:
iboundary = IonicBoundary(
    alpha=1.12,
    softness=0.5,
    ions=ions,
    mode='ionic',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='lowmem',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [ ]:
ions.updating = True
iboundary.update()
ions.updating = False

In [ ]:
print(f"volume  = {iboundary.volume}")
print(f"surface = {iboundary.surface}")

# Initialize an system boundary

In [ ]:
from envyron.boundaries import SystemBoundary

In [ ]:
sboundary = SystemBoundary(
    distance=my_input.solvent.distance,
    spread=my_input.solvent.spread,
    system=system,
    mode='system',
    need_gradient=True,
    need_laplacian=True,
    need_hessian=True,
    deriv_method='chain',
    cores=cores,
    grid=grid,
    label='solvent'
)

In [ ]:
system.updating = True
sboundary.update()
system.updating = False

In [ ]:
print(f"volume  = {sboundary.volume}")
print(f"surface = {sboundary.surface}")